In [1]:
import uproot as ur
import numpy as np
import pandas as pd
import awkward as ak
import pickle

In [2]:
input_file = "/Users/seanbp/Documents/Data/GenII_Prototype_Data/JLab/Beam/63-60/1-63-60-beam.root"

events = ur.open("%s:raw"%(input_file))

In [3]:
# Extract all channel keys
channel_keys = [key for key in events.keys() if key.startswith("ch_")]

# Load all channels into an awkward array dict
channels = {key: events[key].array(library="ak") for key in channel_keys}

# Create a list of awkward arrays for each channel with extra fields
channel_arrays = []
for key, arr in channels.items():
    channel = int(key.split("_")[1])
    arr_with_fields = ak.zip({
        "event": ak.local_index(arr),
        "channel": channel,
        "HG": arr["HG"],
        "LG": arr["LG"],
    })
    channel_arrays.append(arr_with_fields)

# Concatenate all channel arrays
all_data = ak.concatenate(channel_arrays)

# Convert to pandas DataFrame
df = ak.to_dataframe(all_data).reset_index(drop=True)
df = df.sort_values(by=["event", "channel"]).reset_index(drop=True)
df["CAEN_brd"] = df["channel"] // 64
df["CAEN_ch"] = df["channel"] % 64

print(df.head())


   event  channel   HG   LG  CAEN_brd  CAEN_ch
0      0        0  203   99         0        0
1      0        1  173  109         0        1
2      0        2  130  104         0        2
3      0        3  102  120         0        3
4      0        4  139  108         0        4


In [5]:
with open('1-63-60-beam_DF.pkl', 'wb') as f:
    pickle.dump(df, f)